# Fake data creator for the DB Project
This was copied from something I had done for an earlier assignment.

In [1]:
%pip install Faker

## Overall Structure
This section sets up the output for creating insert SQL statement data.

In [140]:
"""
Created on: February 14, 2024

Author: Tuck Williamson

Assignment: 601 Misc\Library
=============
This module creates fake data for testing purposes.
"""
from faker import Faker
import typing
from jinja2 import Template as TP

# This makes results repeatable.
Faker.seed(601)
fake = Faker('en_US')

sqlQuote = "'"
sqteJoin = ", "

INS_TP = TP('''
{% set fields = "" if fields is none else '('+ ', '.join(fields) +')' %}
------------------------------
-- {{table}}
------------------------------
{% if genTransaction %}
BEGIN;            
{% endif %}
{% if genDelete is not false %}DELETE
	FROM {{table}}
	WHERE {{genDelete}} IS NOT NULL;
{% endif %}
INSERT INTO {{table}}{{fields}}
	VALUES {{ elements|join(',\n\t\t')}}
        ;

{% if genTransaction %}
COMMIT;
            
{% endif %}
------------------------------\n\n''')
def create_sql_insert(elements: typing.Iterable[typing.Iterable[str]], table: str,
                      fields: typing.Iterable[str] = None, genDelete: bool|str = False,
                      genTransaction: bool = True) -> str:
    """
    This creates a **SQL INSERT** statement given the elements and table name.
    Fields can also be specified as an array of strings in the order they
    were placed in the tuples in the elements list.

    :param elements: The elements to inject as values into the table. Any elements that
    :param table: The name of the table to insert into.
    :param fields: An optional list of fields names in the order that they were placed in the elements list items.
    :param genDelete: An optional boolean which determines if a delete statement is included before the insert statement.
        If genDelete is a string it is used as the attribute to test for not null.
        If genDelete is True then it uses the first field in fields as the test attribute.
    :return: A string containing the sql for the insert statement.

    """
    retVal = ''
    match genDelete:
        case str(delOnField):
            genDelete = delOnField
        case True:
            assert fields is not None
            genDelete = fields[0]

    elements = ["({row})".format(row = ', '.join([f"'{val}'" if not val.startswith('raw: ') else val[5:] for val in ele])) for ele in elements]
        
    return INS_TP.render(elements=elements, table=table, fields=fields, genDelete=genDelete, genTransaction=genTransaction)
    # return f'\n\n{retVal}{"BEGIN;" if genTransaction else ""}INSERT INTO {table}{"" if fields is None else " (" + ", ".join(fields) + ")"}\n\tVALUES' + ', \n\t\t'.join(
    #     [f"({sqteJoin.join(ele)})" for ele in
    #      [[sqlQuote + ele + sqlQuote if not ele.startswith('raw: ') else ele[5:] for ele in etuple] for etuple in
    #       elements]]) + f';\n{"COMMIT;" if genTransaction else ""}\n'


In [182]:
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot
from numpy import array
import math

posList = [
    'Executive Chef',
    'Manager',
    'Sommelier',
    'Sous Chef',
    'Pastry Chef',
    'Roast Chef',
    'Captain',
    'Bartender',
    'Hosts',
    'Waiter',
    'Runner',
    'Back Waiter',
    'Barback',
    'Busboy',
    ]

# This is just giving weights to the positions based on how common they are.
# plen = len(posList)
# weights = array([float(idx-(plen/2.0)+0.1)/plen for idx in range(len(posList))])
# weights = array([(math.sin(math.pi*val)+1)**3 for val in weights])
# # weights = array([float(idx-(plen/2.0))/plen for idx in range(len(posList))])
# matplotlib.pyplot.plot(posList,weights).show()
weights = array([0,0,0,5,5,10,10,5,10,25,5,5,5,5])/100.0
weights /= sum(weights)
# display(weights)
numEmployees = 90
def getEmp(position = None)->tuple[str, str, str, str, str, str]:
    if position is None:
        position = fake.random_element(OrderedDict([(posList[idx], weights[idx]) for idx in range(len(posList))]))
    return (fake.numerify('#########'), 
            fake.first_name(), 
            fake.last_name(), 
            str(fake.past_date('-5y')), 
            fake.numerify('##########'), 
            position) 
emps = [getEmp() for i in range(numEmployees)]
# An exec chef
emps.append(getEmp(posList[0]))
# num managers
[emps.append(getEmp(posList[1])) for i in range(4)]
# num sommlier
[emps.append(getEmp(posList[2])) for i in range(3)]

# print(f'{"-"*30}\n-- Employees')
print(create_sql_insert(emps, 'thebestdbever.employee', genDelete='SSN'))
# print(f'{"-"*30}\n')

positionIndex = -1
emplIDs = [emp[0] for emp in emps]
empPosMap = {}
for pos in posList:
    #Build a map of positions -> []
    empPosMap[pos] = []

for emp in emps:
    #Build a map of positions -> [employee SSNs]
    empPosMap[emp[positionIndex]].append(emp[0])



------------------------------
-- thebestdbever.employee
------------------------------

BEGIN;            

DELETE
	FROM thebestdbever.employee
	WHERE SSN IS NOT NULL;

INSERT INTO thebestdbever.employee
	VALUES ('538578111', 'Sarah', 'Green', '2023-01-18', '4458312189', 'Busboy'),
		('398049492', 'Casey', 'Smith', '2022-01-06', '5543502978', 'Waiter'),
		('477983760', 'Marcus', 'Jensen', '2020-05-02', '1193379627', 'Hosts'),
		('959475047', 'Kimberly', 'Manning', '2023-08-01', '9617567462', 'Busboy'),
		('975806394', 'Mitchell', 'Gallegos', '2024-03-26', '6680754056', 'Hosts'),
		('030278545', 'Taylor', 'Sanchez', '2023-07-19', '0116168401', 'Waiter'),
		('197281036', 'Katie', 'Newton', '2023-03-26', '2879135858', 'Hosts'),
		('550769928', 'Erik', 'Hicks', '2020-10-13', '9136501797', 'Bartender'),
		('086813571', 'Katie', 'Francis', '2020-05-17', '1269160962', 'Waiter'),
		('764661834', 'Kenneth', 'Smith', '2020-04-13', '0466953829', 'Waiter'),
		('000957839', 'Ryan', 'Blake', '201

In [183]:
import datetime as dt
today = dt.date.today()
dteRange = dt.timedelta(weeks=6)
start = today - dteRange
end = today +  dteRange

def getShift(date, earlyShift:bool = True):
    """This creates value arrays for inserts of Shifts
    """
    return (str(date), '9:30 am' if earlyShift else '5:30 pm', '9h')

days = [start+dt.timedelta(days=i) for i in range((end - start).days)]
earlyShifts = [getShift(day) for day in days if day.weekday() != 0]
lateShifts = [getShift(day, earlyShift=False) for day in days if day.weekday() != 0]

shifts= earlyShifts + lateShifts
print(create_sql_insert(shifts, 'thebestdbever.shift', ['Date', 'StartTime', 'Duration'], True))



------------------------------
-- thebestdbever.shift
------------------------------

BEGIN;            

DELETE
	FROM thebestdbever.shift
	WHERE Date IS NOT NULL;

INSERT INTO thebestdbever.shift(Date, StartTime, Duration)
	VALUES ('2024-02-21', '9:30 am', '9h'),
		('2024-02-22', '9:30 am', '9h'),
		('2024-02-23', '9:30 am', '9h'),
		('2024-02-24', '9:30 am', '9h'),
		('2024-02-25', '9:30 am', '9h'),
		('2024-02-27', '9:30 am', '9h'),
		('2024-02-28', '9:30 am', '9h'),
		('2024-02-29', '9:30 am', '9h'),
		('2024-03-01', '9:30 am', '9h'),
		('2024-03-02', '9:30 am', '9h'),
		('2024-03-03', '9:30 am', '9h'),
		('2024-03-05', '9:30 am', '9h'),
		('2024-03-06', '9:30 am', '9h'),
		('2024-03-07', '9:30 am', '9h'),
		('2024-03-08', '9:30 am', '9h'),
		('2024-03-09', '9:30 am', '9h'),
		('2024-03-10', '9:30 am', '9h'),
		('2024-03-12', '9:30 am', '9h'),
		('2024-03-13', '9:30 am', '9h'),
		('2024-03-14', '9:30 am', '9h'),
		('2024-03-15', '9:30 am', '9h'),
		('2024-03-16', '9:30 am', '9h')

In [184]:


def make_select_for_key(key: str, table: str, column: str = None, value: str = None, where_clause: str = None) -> str:
    """
    This creates a **SQL SELECT** statement from the given parameters.
    These are generally used inside the SQL INSERT statements created from create_sql_insert.
    :param key: The name of the column to select, or a **string** with the name of multiple columns to
        select separated by commas.
    :param table: The name of the table to select from.
    :param column: The name of the column to check against value.
    :param value: The value to check against. See _whereClause_.
    :param where_clause: If whereClause is specified and value and column are None then the whereClause will
        be inserted into the statement as is. This is useful when you need to check multiple values.
    :return: A string with the select statement.
    """
    if value is None and where_clause is None:
        raise Exception("Cannot have value and whereClause both None. Either specify value or whereClause.")
    if column is None and value is not None:
        column = key

    if column is None and value is None:
        return f"raw: (SELECT {key} FROM {table} WHERE {where_clause})"
    else:
        return f"raw: (SELECT {key} FROM {table} WHERE {column} = '{value}')"

# Mgr, host, chef, bartender, waiter
earlyReq = [
    ['Manager'],
    ['Hosts'],
    ['Executive Chef','Sous Chef','Pastry Chef', 'Roast Chef'],
    ['Bartender'],
    ['Captain','Waiter','Sommelier'],]
    # 'Runner',
    # 'Back Waiter',
    # 'Barback',
    # 'Busboy',
    # ]
lateReq = earlyReq.copy()
lateReq.append(('Sommelier'))

def flatten(*args):
    retVal = []
    [retVal.extend(lst) for lst in args]
    return retVal

# display(empPosMap)
earlyReqToSSN = [flatten(*[[(pos, ele) for ele in empPosMap[pos]] for pos in reqPos]) for reqPos in earlyReq]
worksOnShift = flatten(*[[(make_select_for_key("shiftid", 'shift', where_clause=f"Date='{shift[0]}' AND StartTime='{shift[1]}'"), (worker:=fake.random_element(ssns))[1], worker[0]) for ssns in earlyReqToSSN] for shift in earlyShifts])
# display(worksOnShift)
print(create_sql_insert(worksOnShift, 'works_on_shift', genDelete='shiftid'))



------------------------------
-- works_on_shift
------------------------------

BEGIN;            

DELETE
	FROM works_on_shift
	WHERE shiftid IS NOT NULL;

INSERT INTO works_on_shift
	VALUES ((SELECT shiftid FROM shift WHERE Date='2024-02-21' AND StartTime='9:30 am'), '212884559', 'Manager'),
		((SELECT shiftid FROM shift WHERE Date='2024-02-21' AND StartTime='9:30 am'), '351060262', 'Hosts'),
		((SELECT shiftid FROM shift WHERE Date='2024-02-21' AND StartTime='9:30 am'), '426599279', 'Roast Chef'),
		((SELECT shiftid FROM shift WHERE Date='2024-02-21' AND StartTime='9:30 am'), '314089880', 'Bartender'),
		((SELECT shiftid FROM shift WHERE Date='2024-02-21' AND StartTime='9:30 am'), '038756607', 'Waiter'),
		((SELECT shiftid FROM shift WHERE Date='2024-02-22' AND StartTime='9:30 am'), '045312244', 'Manager'),
		((SELECT shiftid FROM shift WHERE Date='2024-02-22' AND StartTime='9:30 am'), '975806394', 'Hosts'),
		((SELECT shiftid FROM shift WHERE Date='2024-02-22' AND StartTime='9:3